## 

### Assessment Basket ball Tracking

In [61]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time

# Path to the video file
video_path = r"WHATSAAP ASSIGNMENT.mp4"
# HSV color range for detecting yellow
yellowLower = (20, 100, 100)
yellowUpper = (30, 255, 255)
# Deque to store the tracked points
pts = deque()

# Initialize the video stream and background subtractor
vs = cv2.VideoCapture(video_path)
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Parameters for dribble detection
dribble_count = 0
is_dribbling = False
dribble_frame_count = 0
dribble_threshold = 2
dribble_speeds = []
prev_center = None

# Variables for additional parameters
total_dribble_attempts = 0
dribble_efficiency = 0
dribble_directions = []

# To iterate through each a loop so that to process each frame of the video
while True:
    #returns the flag whether successfully returned frame or not and frame
    ret, frame = vs.read()
    if not ret:
        break
    #subtracts the background
    fg_mask = bg_subtractor.apply(frame)

    #resizing the frame 
    frame = imutils.resize(frame, width=500)
    #applying gaussian blur such that kernersize_width,ksize_height and should be odd, standard_dev
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    
    ##changing to the Hue,saturation and values, so that to extract yellow patterns
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    #binary mask where the pixels falling in range yellowLower and yellowUpper &isolates the yellow color regions
    mask = cv2.inRange(hsv, yellowLower, yellowUpper)
    #Erosion is a morphological operation that shrinks the white regions in the binary mask
    mask = cv2.erode(mask, None, iterations=2) #iterations parameter: number of times erosion is applied
    #dilation is often used to reconnect broken parts of an object or to fill in gaps in the object boundaries.
    mask = cv2.dilate(mask, None, iterations=2)

    # Find contours in the mask, to detect the basketball
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    center = None

    if len(contours) > 0:
        # Find the largest contour and its centroid
        c = max(contours, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        if radius > 10:
            # Draw the circle and centroid on the frame
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 0), 2)
            cv2.circle(frame, center, 5, (100, 100, 100), -1)

            # Update dribble attempts count
            total_dribble_attempts += 1

            # Check for dribble detection,if there's a significant change in the position of the basketball.
            if prev_center is not None:
               #if movement exceeds certain threshold 49px in both x & y directions,it's considered a dribble.
                if abs(center[0] - prev_center[0]) > 49 or abs(center[1] - prev_center[1]) > 49:
                    dribble_frame_count += 1
                    #checks if the count of frames where a dribble is detected exceeds a predefined threshold
                    if dribble_frame_count >= dribble_threshold:
                        if not is_dribbling:
                            dribble_count += 1
                             #to indicate that a dribble is currently being detected.
                            is_dribbling = True
                            #Records the start time of the detected dribble
                            start_time = time.time()
                            # Record dribble direction
                            dribble_directions.append((center[0] - prev_center[0], center[1] - prev_center[1]))
                    else:
                        is_dribbling = False
                else:
                    dribble_frame_count = 0
                    if is_dribbling:
                        end_time = time.time()
                        time_taken = end_time - start_time
                        if time_taken > 0:
                            distance = np.sqrt((center[0] - prev_center[0])**2 + (center[1] - prev_center[1])**2)
                            dribble_speed = (distance / time_taken)/ 1000
                            dribble_speeds.append(dribble_speed)
                        #if not condition met of dribbling
                        is_dribbling = False
            #update the center for next iteration loop
            prev_center = center

    pts.appendleft(center)

    # Calculate dribble efficiency
    if total_dribble_attempts > 0:
        dribble_efficiency = dribble_count / total_dribble_attempts

    # displays the dribble count on the frame
    cv2.putText(frame, f'Dribbles: {dribble_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
    # displays the dribble efficiency on the frame
    cv2.putText(frame, f'Dribble Efficiency: {dribble_efficiency:.2f}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
    
    # Show the frame
    cv2.imshow("my_frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # Check for quit key
    if cv2.waitKey(25) & 0xFF== ord("q"):
        break
cap.release()
cv2.destroyAllWindows()
# Print the total dribble count and average dribble speed
print('Total Dribbles: ', dribble_count)
print(f"Dribble_efficiency{dribble_efficiency}")
if dribble_speeds:
    print('Average Dribble Speed: ', sum(dribble_speeds) / len(dribble_speeds))
else:
    print('No dribble speeds recorded.')

Total Dribbles:  22
Dribble_efficiency0.07333333333333333
Average Dribble Speed:  0.4102920885155624
